In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import os
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

In [ ]:
data_path = "/content/company_sec.csv"

In [ ]:
def data_loader(data_path):
    df = pd.read_csv(data_path)
    return df

In [ ]:
def data_cleaning(data):
    """
    Cleans specified columns in the DataFrame:
    - Strips whitespace from column names.
    - Removes commas from numerical string values.
    - Replaces dash-like strings and 'NaN' with actual NaN values.
    - Converts cleaned strings to float.

    Parameters:
    data (pd.DataFrame): Input DataFrame with potentially messy numeric columns.

    Returns:
    pd.DataFrame: Cleaned DataFrame with specified columns converted to float.
    """
    data.columns = data.columns.str.strip()

    cols = ['Feature4', 'Feature5', 'Feature6', 'Feature7',
            'Feature9', 'Target 1', 'Target 2', 'Target 3']

    for col in cols:
        data[col] = (
            data[col]
            .astype(str)
            .str.replace(',', '', regex=False)
            .replace(['-', ' - ', ' -   ', '  -', 'NaN'], np.nan)
            .astype(float)
        )

    return data

In [ ]:
df = data_loader(data_path)
df = data_cleaning(df)

In [ ]:
feature_cols = [f'Feature{i}' for i in range(1, 29)]


In [ ]:
ROLL_WINDOW = 2  # back and forward years
TOP_K_NEIGHBORS = 3
ALPHA = 0.5
BETA = 0.5

# Create timestamp for log files
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
imputation_log_file = f"imputation_log_{timestamp}.csv"
hybrid_details_file = f"hybrid_imputation_details_{timestamp}.csv"
summary_plot_file = f"imputation_summary_{timestamp}.png"

# Log columns
log_columns = ["Company", "Year", "Feature", "Method", "Value", "Details"]
hybrid_columns = ["Company", "Year", "Feature", "Target_Value", "Neighbor_Company",
                 "Neighbor_Year", "Neighbor_Value", "Sector_Similarity",
                 "Feature_Similarity", "Total_Similarity"]

# Initialize logs
imputation_logs = []
hybrid_details = []

print("Starting imputation process...")

# --- Step 1: Filter companies with ≥ 5 unique years ---
print("Filtering companies with at least 5 unique years...")
valid_companies = df['Company'].value_counts()
valid_companies = valid_companies[valid_companies >= 5].index
df_filtered = df[df['Company'].isin(valid_companies)].reset_index(drop=True)
print(f"Filtered from {df['Company'].nunique()} to {df_filtered['Company'].nunique()} companies")

# --- Step 1.5: Compute total and fillable missing values ---
print("Computing missing value statistics...")
total_missing_all = df[feature_cols].isna().sum().sum()
total_missing_filtered = df_filtered[feature_cols].isna().sum().sum()

potentially_fillable_missing = 0
for company, group in df_filtered.groupby("Company"):
    for feature in feature_cols:
        if group[feature].isna().all():
            continue
        potentially_fillable_missing += group[feature].isna().sum()
# Count missing values that cannot be filled due to entire feature column being NaN for a company
non_fillable_missing = 0
for company, group in df_filtered.groupby("Company"):
    for feature in feature_cols:
        if group[feature].isna().all():
            non_fillable_missing += len(group)



print(f"Total missing values in all data: {total_missing_all}")
print(f"Total missing values in filtered data (eligible companies): {total_missing_filtered}")
print(f"Potentially fillable missing values: {potentially_fillable_missing}")
print(f"Non-fillable missing values (entire feature column is NaN for a company): {non_fillable_missing}")
# --- Step 2: Prepare sector embeddings ---
print("Preparing sector embeddings...")
unique_sectors = df_filtered['Sector'].dropna().unique()
model = SentenceTransformer('all-MiniLM-L6-v2')
sector_embeddings = model.encode(unique_sectors, show_progress_bar=True)
sector_embedding_df = pd.DataFrame(sector_embeddings, index=unique_sectors)
sector_embedding_df.index.name = "Sector"
sector_embedding_df = sector_embedding_df.reset_index()
sector_embedding_map = dict(zip(sector_embedding_df["Sector"], sector_embedding_df.drop(columns="Sector").values))
print(f"Created embeddings for {len(unique_sectors)} unique sectors")

# --- Step 3: Impute per company ---
imputed_frames = []
company_groups = list(df_filtered.groupby('Company'))
total_imputed_values = 0
total_mean_imputed = 0
total_hybrid_imputed = 0

for company, group in tqdm(company_groups, desc="Processing companies"):
    if group['Year'].nunique() < 5:
        continue

    group = group.sort_values(by='Year').copy()
    sector = group['Sector'].iloc[0]

    if pd.isna(sector) or sector not in sector_embedding_map:
        imputed_frames.append(group)
        continue

    for feature in tqdm(feature_cols, desc=f"Features for {company}", leave=False):
        if group[feature].isna().all():
            continue

        missing_indices = group.index[group[feature].isna()]

        for idx in tqdm(missing_indices, desc=f"Missing values in {feature}", leave=False):
            year = group.loc[idx, 'Year']
            lower_year = year - ROLL_WINDOW
            upper_year = year + ROLL_WINDOW
            neighbor_values = group[
                (group['Year'].between(lower_year, upper_year)) &
                (group[feature].notna())
            ][feature]

            if len(neighbor_values) >= 2:
                imputed_value = neighbor_values.mean()
                group.at[idx, feature] = imputed_value
                total_mean_imputed += 1
                imputation_logs.append({
                    "Company": company,
                    "Year": year,
                    "Feature": feature,
                    "Method": "Mean",
                    "Value": imputed_value,
                    "Details": f"Used years: {neighbor_values.index.tolist()}"
                })
            else:
                target_row = group.loc[idx]
                available_feats = group.loc[idx, feature_cols].dropna().index.tolist()
                candidates = df_filtered[
                    (df_filtered['Company'] != company) &
                    (df_filtered['Year'].between(year - 3, year + 3)) &
                    (df_filtered[feature].notna())
                ]

                sims = []
                candidate_details = []
                for _, cand in candidates.iterrows():
                    cand_sector = cand['Sector']
                    if pd.isna(cand_sector) or cand_sector not in sector_embedding_map:
                        continue

                    sec_sim = cosine_similarity(
                        sector_embedding_map[sector].reshape(1, -1),
                        sector_embedding_map[cand_sector].reshape(1, -1)
                    )[0][0]

                    common_feats = [f for f in available_feats if pd.notna(cand[f])]
                    if not common_feats:
                        continue

                    target_vals = target_row[common_feats].values.astype(float)
                    cand_vals = cand[common_feats].values.astype(float)
                    feat_dist = np.linalg.norm(target_vals - cand_vals)
                    feat_sim = 1 / (1 + feat_dist)

                    total_sim = ALPHA * sec_sim + BETA * feat_sim
                    sims.append((cand[feature], total_sim))
                    candidate_details.append({
                        "Company": company,
                        "Year": year,
                        "Feature": feature,
                        "Target_Value": np.nan,
                        "Neighbor_Company": cand['Company'],
                        "Neighbor_Year": cand['Year'],
                        "Neighbor_Value": cand[feature],
                        "Sector_Similarity": sec_sim,
                        "Feature_Similarity": feat_sim,
                        "Total_Similarity": total_sim
                    })

                if sims:
                    top_k = sorted(sims, key=lambda x: x[1], reverse=True)[:TOP_K_NEIGHBORS]
                    values, weights = zip(*top_k)
                    imputed_value = np.average(values, weights=weights)
                    group.at[idx, feature] = imputed_value
                    total_hybrid_imputed += 1
                    top_indices = sorted(range(len(sims)), key=lambda i: sims[i][1], reverse=True)[:TOP_K_NEIGHBORS]
                    for i in top_indices:
                        hybrid_details.append(candidate_details[i])
                        hybrid_details[-1]["Target_Value"] = imputed_value
                    neighbor_info = ", ".join([
                        f"{candidate_details[i]['Neighbor_Company']} (Y:{candidate_details[i]['Neighbor_Year']}, S:{candidate_details[i]['Total_Similarity']:.3f})"
                        for i in top_indices
                    ])
                    imputation_logs.append({
                        "Company": company,
                        "Year": year,
                        "Feature": feature,
                        "Method": "Hybrid",
                        "Value": imputed_value,
                        "Details": f"Used neighbors: {neighbor_info}"
                    })

            total_imputed_values += 1

    imputed_frames.append(group)

# --- Final result and logs ---
df_imputed_final = pd.concat(imputed_frames).reset_index(drop=True)

print(f"Writing imputation logs to {imputation_log_file}...")
pd.DataFrame(imputation_logs, columns=log_columns).to_csv(imputation_log_file, index=False)

print(f"Writing hybrid imputation details to {hybrid_details_file}...")
pd.DataFrame(hybrid_details, columns=hybrid_columns).to_csv(hybrid_details_file, index=False)

# --- Summary Plot ---
print(f"Generating summary plot: {summary_plot_file}...")
plt.figure(figsize=(8, 5))
sns.barplot(
    x=["Mean Imputation", "Hybrid Imputation"],
    y=[total_mean_imputed, total_hybrid_imputed],
    palette="pastel"
)
plt.title("Imputation Method Distribution")
plt.ylabel("Number of Values Imputed")
plt.tight_layout()
plt.savefig(summary_plot_file)

print("\nImputation Summary:")
print(f"Total imputed values: {total_imputed_values}")
print(f"Values imputed using mean: {total_mean_imputed} ({total_mean_imputed/total_imputed_values*100:.1f}%)")
print(f"Values imputed using hybrid approach: {total_hybrid_imputed} ({total_hybrid_imputed/total_imputed_values*100:.1f}%)")

Starting imputation process...
Filtering companies with at least 5 unique years...
Filtered from 2796 to 1566 companies
Computing missing value statistics...
Total missing values in all data: 19108
Total missing values in filtered data (eligible companies): 15134
Potentially fillable missing values: 14107
Non-fillable missing values (entire feature column is NaN for a company): 1027
Preparing sector embeddings...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/14 [00:00<?, ?it/s]

Created embeddings for 425 unique sectors


Streaming output truncated to the last 5000 lines.

                                              

Missing values in Feature4: 0it [00:00, ?it/s]

                                              

Missing values in Feature5:   0%|          | 0/1 [00:00<?, ?it/s]

                                                                 

Missing values in Feature6:   0%|          | 0/1 [00:00<?, ?it/s]

                                                                 

Missing values in Feature7:   0%|          | 0/1 [00:00<?, ?it/s]

                                                                 

Missing values in Feature8: 0it [00:00, ?it/s]

                                              

Missing values in Feature9: 0it [00:00, ?it/s]

                                              

Missing values in Feature10:   0%|          | 0/1 [00:00<?, ?it/s]

                                                                  

Missing values in Feature11:   0%|          | 0/1 [00:00<?, ?it/s]

      

In [ ]:
df = df_imputed_final

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Function to create sector embeddings
def create_sector_embeddings(df):
    model = SentenceTransformer('all-MiniLM-L6-v2')
    unique_sectors = df['Sector'].unique()
    sector_embeddings = {}

    print("Creating embeddings for each sector...")
    for sector in tqdm(unique_sectors, desc="Sector Embeddings"):
        sector_embeddings[sector] = model.encode(sector)

    company_year_to_sector_embedding = {}
    print("Mapping embeddings to company-year pairs...")
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Mapping Embeddings"):
        key = (row['Company'], row['Year'])
        company_year_to_sector_embedding[key] = sector_embeddings[row['Sector']]

    return company_year_to_sector_embedding

# Identify companies with <5 years of data
def identify_companies_to_impute(df):
    counts = df.groupby('Company').size().reset_index(name='count')
    return set(counts[counts['count'] < 5]['Company'])

# KNN imputation with sector and ±3 year filter
def custom_knn_impute(df, feature_cols, companies_to_impute, company_year_to_sector_embedding,
                      sector_weight=0.3, n_neighbors=8):
    df_imputed = df.copy()
    neighbors_log = []

    print("Scaling features...")
    scaler = StandardScaler()
    df_scaled = df.copy()
    df_scaled[feature_cols] = scaler.fit_transform(df[feature_cols].fillna(0))

    total_imputed = 0
    rows_to_impute = [idx for idx, row in df.iterrows()
                      if row['Company'] in companies_to_impute and any(pd.isna(row[col]) for col in feature_cols)]

    print(f"Found {len(rows_to_impute)} rows that need imputation...")

    for idx in tqdm(rows_to_impute, desc="Imputing Rows"):
        row = df.loc[idx]
        company, year = row['Company'], row['Year']
        missing_cols = [col for col in feature_cols if pd.isna(row[col])]

        if not missing_cols:
            continue

        key = (company, year)
        current_sector_embedding = company_year_to_sector_embedding.get(key)
        if current_sector_embedding is None:
            continue

        similarities = []

        for other_idx, other_row in df.iterrows():
            if idx == other_idx:
                continue

            other_company = other_row['Company']
            other_year = other_row['Year']

            # Only consider rows within ±3 years
            if abs(other_year - year) > 3:
                continue

            if any(pd.isna(other_row[col]) for col in missing_cols):
                continue

            valid_features = [col for col in feature_cols
                              if not pd.isna(row[col]) and not pd.isna(other_row[col])]

            if not valid_features:
                continue

            feature_distance = np.linalg.norm(
                df_scaled.loc[idx, valid_features].values -
                df_scaled.loc[other_idx, valid_features].values
            )
            feature_similarity = 1 / (1 + feature_distance)

            other_key = (other_company, other_year)
            other_sector_embedding = company_year_to_sector_embedding.get(other_key)
            if other_sector_embedding is None:
                continue

            sector_similarity = cosine_similarity(
                [current_sector_embedding],
                [other_sector_embedding]
            )[0][0]

            weighted_similarity = (1 - sector_weight) * feature_similarity + sector_weight * sector_similarity
            similarities.append((other_idx, weighted_similarity, other_company, other_year))

        similarities.sort(key=lambda x: x[1], reverse=True)
        top_neighbors = similarities[:n_neighbors]

        if not top_neighbors:
            continue

        for col in missing_cols:
            neighbor_values = [df.loc[n_idx, col] for n_idx, _, _, _ in top_neighbors]
            neighbor_weights = [sim for _, sim, _, _ in top_neighbors]

            imputed_value = np.average(neighbor_values, weights=neighbor_weights)
            df_imputed.loc[idx, col] = imputed_value
            total_imputed += 1

            neighbors_log.append({
                'Company': company,
                'Year': year,
                'Feature': col,
                'Imputed_Value': imputed_value,
                'Neighbors': ', '.join([f"{n_company} ({n_year})" for _, _, n_company, n_year in top_neighbors]),
                'Neighbor_Weights': ', '.join([f"{weight:.4f}" for _, weight, _, _ in top_neighbors])
            })

    print(f"Total values imputed: {total_imputed}")
    return df_imputed, pd.DataFrame(neighbors_log)

# Master function to run everything
def run_imputation(df, sector_weight=0.5, n_neighbors=6, output_dir='./'):
    import os
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    print(f"Starting imputation on dataframe with shape: {df.shape}")
    print(f"Initial missing values: {df.iloc[:, 3:].isna().sum().sum()}")

    feature_cols = [col for col in df.columns if col.startswith('Feature')]
    print(f"Detected {len(feature_cols)} feature columns.")

    print("Creating sector embeddings...")
    company_year_to_sector_embedding = create_sector_embeddings(df)

    print("Identifying companies with fewer than 5 years of data...")
    companies_to_impute = identify_companies_to_impute(df)
    print(f"Found {len(companies_to_impute)} such companies.")

    print("Running custom KNN imputation...")
    df_imputed, neighbors_log_df = custom_knn_impute(
        df,
        feature_cols,
        companies_to_impute,
        company_year_to_sector_embedding,
        sector_weight,
        n_neighbors
    )

    imputed_path = os.path.join(output_dir, 'imputed_data.csv')
    log_path = os.path.join(output_dir, 'neighbors_log.csv')

    df_imputed.to_csv(imputed_path, index=False)
    neighbors_log_df.to_csv(log_path, index=False)

    print(f"\nImputed data saved to: {imputed_path}")
    print(f"Neighbors log saved to: {log_path}")

    print("\nImputation Summary:")
    print(f"Original missing values: {df[feature_cols].isna().sum().sum()}")
    print(f"Remaining missing values: {df_imputed[feature_cols].isna().sum().sum()}")
    print(f"Values imputed: {df[feature_cols].isna().sum().sum() - df_imputed[feature_cols].isna().sum().sum()}")
    print(f"Number of imputation operations logged: {len(neighbors_log_df)}")

    if len(neighbors_log_df) > 0:
        print("\nSample of neighbors_log.csv:")
        print(neighbors_log_df.head())

    return df_imputed, neighbors_log_df



run_imputation(df, sector_weight=0.5, n_neighbors=6, output_dir='./')

In [ ]:
df1 = run_imputation(df, sector_weight=0.5, n_neighbors=6, output_dir='./')

In [ ]:
# Step 1: Count occurrences of each company in df_impute
company_counts = df_imputed_final['Company'].value_counts()

# Step 2: Identify companies with fewer than 5 records
underrepresented_companies = company_counts[company_counts < 5].index.tolist()

print(f"🔍 Companies with <5 entries in imputed data: {len(underrepresented_companies)}")

# Step 3: Filter out these companies from df_imputed_final
df_imputed_final_filtered = df_imputed_final[~df_imputed_final['Company'].isin(underrepresented_companies)]

# Step 4: Get corresponding rows from df1 for these underrepresented companies
df1_rows_to_add = df1[df1['Company'].isin(underrepresented_companies)]

# Step 5: Concatenate both parts to form the final DataFrame
df_final = pd.concat([df_imputed_final_filtered, df1_rows_to_add], ignore_index=True)

# Optional: Sort and reset index
df_final = df_final.sort_values(by=['Company', 'Year']).reset_index(drop=True)
print(f"✅ Final shape after merging: {df_final.shape}")

In [ ]:
df_final2=df_final.copy()
print("\nStarting final hybrid imputation for previously unfillable values...")

remaining_missing = df_final2[feature_cols].isna()
final_hybrid_added = 0

# Go row-by-row where there’s at least one NaN value
for idx, row in tqdm(df_final2[remaining_missing.any(axis=1)].iterrows(), total=remaining_missing.any(axis=1).sum(), desc="Final Hybrid Imputation"):
    company = row['Company']
    year = row['Year']
    sector = row['Sector']
    if pd.isna(sector) or sector not in sector_embedding_map:
        continue

    # For each feature still missing
    for feature in feature_cols:
        if not pd.isna(row[feature]):
            continue

        # Find which features have values (used for feature similarity)
        available_feats = row[feature_cols].dropna().index.tolist()
        if not available_feats:
            continue  # can't calculate similarity without any overlap

        # Find candidate rows from other companies, within ±5 years, with the feature present
        candidates = df_filtered[
            (df_filtered['Company'] != company) &
            (df_filtered['Year'].between(year - 3, year + 3)) &
            (df_filtered[feature].notna())
        ]

        sims = []
        candidate_details = []

        for _, cand in candidates.iterrows():
            cand_sector = cand['Sector']
            if pd.isna(cand_sector) or cand_sector not in sector_embedding_map:
                continue

            # 1. Sector similarity
            sec_sim = cosine_similarity(
                sector_embedding_map[sector].reshape(1, -1),
                sector_embedding_map[cand_sector].reshape(1, -1)
            )[0][0]

            # 2. Feature similarity on overlapping features
            common_feats = [f for f in available_feats if pd.notna(cand[f])]
            if not common_feats:
                continue

            target_vals = row[common_feats].values.astype(float)
            cand_vals = cand[common_feats].values.astype(float)
            feat_dist = np.linalg.norm(target_vals - cand_vals)
            feat_sim = 1 / (1 + feat_dist)  # convert distance to similarity

            # 3. Hybrid similarity score
            total_sim = ALPHA * sec_sim + BETA * feat_sim
            sims.append((cand[feature], total_sim))

            # Log the neighbor details
            candidate_details.append({
                "Company": company,
                "Year": year,
                "Feature": feature,
                "Target_Value": np.nan,
                "Neighbor_Company": cand['Company'],
                "Neighbor_Year": cand['Year'],
                "Neighbor_Value": cand[feature],
                "Sector_Similarity": sec_sim,
                "Feature_Similarity": feat_sim,
                "Total_Similarity": total_sim
            })

        # If we found at least one neighbor
        if sims:
            top_k = sorted(sims, key=lambda x: x[1], reverse=True)[:5]
            values, weights = zip(*top_k)
            imputed_value = np.average(values, weights=weights)

            # Update the dataframe
            df_final2.at[idx, feature] = imputed_value
            final_hybrid_added += 1

            # Save top 5 neighbor details
            top_indices = sorted(range(len(sims)), key=lambda i: sims[i][1], reverse=True)[:5]
            for i in top_indices:
                hybrid_details.append(candidate_details[i])
                hybrid_details[-1]["Target_Value"] = imputed_value

            neighbor_info = ", ".join([
                f"{candidate_details[i]['Neighbor_Company']} (Y:{candidate_details[i]['Neighbor_Year']}, S:{candidate_details[i]['Total_Similarity']:.3f})"
                for i in top_indices
            ])
            imputation_logs.append({
                "Company": company,
                "Year": year,
                "Feature": feature,
                "Method": "Final-Hybrid",
                "Value": imputed_value,
                "Details": f"Used neighbors: {neighbor_info}"
            })

print(f"Final hybrid-imputed additional values: {final_hybrid_added}")
# Save full neighbor log
neighbors_log_file = f"neighbors_log_{timestamp}.csv"
print(f"Saving full hybrid neighbor log to {neighbors_log_file}...")
pd.DataFrame(hybrid_details, columns=hybrid_columns).to_csv(neighbors_log_file, index=False)


In [ ]:
df_final=df_final2

In [ ]:
df = df_final

In [ ]:
df['Target 1'] = pd.to_numeric(df['Target 1'], errors='coerce')
df['Target 2'] = pd.to_numeric(df['Target 2'], errors='coerce')
df['Target 3'] = pd.to_numeric(df['Target 3'], errors='coerce')

# Remove rows with Year >= 2022
df = df[df['Year'] < 2022]

# Optionally, you can reset the index after filtering
df = df.reset_index(drop=True)

In [ ]:

df=df
# --- CONFIGURATION ---
feature_cols = [f'Feature{i}' for i in range(1, 29)]
target_cols = ['Target 1', 'Target 2', 'Target 3']
ROLL_WINDOW = 2
YEAR_WINDOW = 3
TOP_K_NEIGHBORS = 3
ALPHA = 0.5
BETA = 0.5

# === CLEANUP: Remove companies with only 1–2 years AND all 3 targets missing ===
def clean_dataset(df):
    grouped = df.groupby("Company")
    clean_rows = []
    for name, group in grouped:
        if group.shape[0] <= 2 and group[target_cols].isna().all(axis=1).all():
            continue  # skip entire group
        clean_rows.append(group)
    return pd.concat(clean_rows, axis=0).reset_index(drop=True)

# df = pd.read_csv("your_data.csv")  # Load your dataset

# Step 1: Clean dataset
print("Cleaning dataset...")
df = clean_dataset(df)

# === TDM ANALYSIS: Calculate Total Data Missing and potential fillable values ===
def tdm_analysis(df, target_cols):
    print("\n=== TOTAL DATA MISSING (TDM) ANALYSIS ===")

    # Overall statistics for each target column
    total_records = len(df)
    missing_stats = {}

    for col in target_cols:
        missing_count = df[col].isna().sum()
        missing_pct = (missing_count / total_records) * 100
        missing_stats[col] = {
            "Total Records": total_records,
            "Missing Values": missing_count,
            "Missing Percentage": f"{missing_pct:.2f}%"
        }
        print(f"\n{col}:")
        print(f"  Total Records: {total_records}")
        print(f"  Missing Values: {missing_count} ({missing_pct:.2f}%)")

    # Calculate potentially fillable values through different methods
    print("\nEstimating fillable values for each target:")

    # Group by company for rolling mean analysis
    fillable_counts = {col: {"Rolling Mean": 0, "Hybrid": 0, "Fallback": 0, "Unfillable": 0} for col in target_cols}

    for company, group in df.groupby("Company"):
        group = group.sort_values(by='Year')
        for target_col in target_cols:
            missing_indices = group.index[group[target_col].isna()]

            for idx in missing_indices:
                row = group.loc[idx]
                year = row['Year']

                # Check if Rolling Mean is possible
                neighbors = group[(group['Year'].between(year - ROLL_WINDOW, year + ROLL_WINDOW)) &
                                   (group[target_col].notna())]
                if len(neighbors) >= 2:
                    fillable_counts[target_col]["Rolling Mean"] += 1
                    continue

                # Check if Hybrid approach might work
                available_feats = row[feature_cols].dropna().index.tolist()
                if available_feats and not pd.isna(row['Sector']):
                    fillable_counts[target_col]["Hybrid"] += 1
                    continue

                # Check if Fallback method would work
                if group[target_col].notna().sum() > 0:
                    fillable_counts[target_col]["Fallback"] += 1
                else:
                    fillable_counts[target_col]["Unfillable"] += 1

    # Print results
    for col in target_cols:
        total_fillable = sum(fillable_counts[col].values()) - fillable_counts[col]["Unfillable"]
        fill_percentage = (total_fillable / missing_stats[col]["Missing Values"]) * 100 if missing_stats[col]["Missing Values"] > 0 else 0

        print(f"\n{col} - Estimated fillable values:")
        print(f"  Rolling Mean: {fillable_counts[col]['Rolling Mean']}")
        print(f"  Hybrid Method: {fillable_counts[col]['Hybrid']}")
        print(f"  Fallback Method: {fillable_counts[col]['Fallback']}")
        print(f"  Unfillable: {fillable_counts[col]['Unfillable']}")
        print(f"  Total Fillable: {total_fillable} of {missing_stats[col]['Missing Values']} ({fill_percentage:.2f}%)")

    # Plot the distribution of methods
    fig, axes = plt.subplots(1, len(target_cols), figsize=(15, 5))
    for i, col in enumerate(target_cols):
        data = fillable_counts[col].copy()
        labels = list(data.keys())
        values = list(data.values())
        axes[i].pie(values, labels=labels, autopct='%1.1f%%', startangle=90)
        axes[i].set_title(f'Fillable Distribution - {col}')

    plt.tight_layout()
    plt.show()

    return missing_stats, fillable_counts

# Step 2: Compute sector embeddings
print("Preparing sector embeddings...")
unique_sectors = df['Sector'].dropna().unique()
sector_model = SentenceTransformer('all-MiniLM-L6-v2')
sector_embeddings = sector_model.encode(unique_sectors, show_progress_bar=True)
sector_embedding_map = dict(zip(unique_sectors, sector_embeddings))

# Step 3: Run TDM Analysis before imputation
missing_stats, fillable_counts = tdm_analysis(df, target_cols)

# Step 4: Imputation Process
print("\n=== BEGINNING IMPUTATION PROCESS ===")
print("Only proceeding with imputation after TDM analysis...")

# Logging setup
method_counts = {"Rolling Mean": 0, "Hybrid": 0, "Fallback": 0}
imputation_logs = []

def impute_target(row, target_col, company_df, global_df):
    year = row['Year']
    # First try Rolling Mean approach
    neighbors = company_df[(company_df['Year'].between(year - ROLL_WINDOW, year + ROLL_WINDOW)) &
                           (company_df[target_col].notna())]
    if len(neighbors) >= 2:
        return neighbors[target_col].mean(), "Rolling Mean"

    # Try Hybrid approach
    candidates = global_df[(global_df['Year'].between(year - YEAR_WINDOW, year + YEAR_WINDOW)) &
                           (global_df[target_col].notna())]
    sims = []
    sector = row['Sector']
    sector_emb = sector_embedding_map.get(sector, None)
    available_feats = row[feature_cols].dropna().index.tolist()

    for _, cand in candidates.iterrows():
        cand_sector = cand['Sector']
        cand_emb = sector_embedding_map.get(cand_sector, None)
        sec_sim = cosine_similarity([sector_emb], [cand_emb])[0][0] if sector_emb is not None and cand_emb is not None else 0
        common_feats = [f for f in available_feats if pd.notna(cand[f])]

        if not common_feats:
            continue

        a = row[common_feats].values.astype(float)
        b = cand[common_feats].values.astype(float)
        feat_sim = 1 / (1 + np.linalg.norm(a - b))
        sims.append((cand[target_col], ALPHA * sec_sim + BETA * feat_sim))

    if sims:
        sims = sorted(sims, key=lambda x: x[1], reverse=True)[:TOP_K_NEIGHBORS]
        values, weights = zip(*sims)
        return np.average(values, weights=weights), "Hybrid"

    # Fallback method
    return company_df[target_col].mean() if company_df[target_col].notna().sum() > 0 else np.nan, "Fallback"

# Group and process with tqdm
final_frames = []
global_df = df.copy()
companies = list(df.groupby("Company"))

# Filter out rows with Year >= 2022 before imputation
df = df[df['Year'] < 2022]
global_df = global_df[global_df['Year'] < 2022]

# Re-create companies list after filtering
companies = list(df.groupby("Company"))

# Display remaining count after filtering
print(f"Records after filtering Year < 2022: {len(df)}")

for company, group in tqdm(companies, desc="Processing Companies"):
    group = group.sort_values(by='Year').copy()
    for target_col in target_cols:
        # Convert target columns to numeric before imputation
        group[target_col] = pd.to_numeric(group[target_col], errors='coerce')

        missing_mask = group[target_col].isna()
        for idx in group.index[missing_mask]:
            row = group.loc[idx]
            imputed_val, method = impute_target(row, target_col, group, global_df)
            group.at[idx, target_col] = imputed_val
            method_counts[method] += 1
            imputation_logs.append({
                "Company": company,
                "Year": row['Year'],
                "Target": target_col,
                "Value": imputed_val,
                "Method": method
            })
    final_frames.append(group)

# Combine and show summary
df_imputed = pd.concat(final_frames).reset_index(drop=True)
print("\nImputation Summary:")
for method, count in method_counts.items():
    print(f"{method}: {count}")

# Plot summary of methods used
plt.figure(figsize=(8, 5))
sns.barplot(x=list(method_counts.keys()), y=list(method_counts.values()), palette="Set2")
plt.title("Imputation Method Counts")
plt.tight_layout()
plt.show()

# Compare before and after
print("\n=== BEFORE vs AFTER IMPUTATION ===")
for col in target_cols:
    before = missing_stats[col]["Missing Values"]
    after = df_imputed[col].isna().sum()
    filled = before - after
    fill_rate = (filled / before) * 100 if before > 0 else 0

    print(f"{col}:")
    print(f"  Missing Before: {before}")
    print(f"  Missing After: {after}")
    print(f"  Values Filled: {filled} ({fill_rate:.2f}%)")

In [ ]:
final_imputed_data = df_imputed